In [1]:
import cv2
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd
import time

from datetime import datetime
import shutil

/Users/callinswitzer/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
os.chdir("/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning")

In [3]:
# get paths for videos

vidPaths = []

for path, subdirs, files in os.walk(os.getcwd()):
    for name in files:
        if name.endswith(".mp4"):
            vidPaths.append(os.path.join(path, name))
vidPaths

['/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning/BeeBlue2_28Nov2016_Hive4_high_compressed/BeeBlue2_28Nov2016_Hive4_high_compressed.mp4',
 '/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning/BeeBlue3_28Nov2016_Hive4_high_compressed/BeeBlue3_28Nov2016_Hive4_high_compressed.mp4',
 '/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning/BeeBlue4_28Nov2016_Hive4_high_compressed/BeeBlue4_28Nov2016_Hive4_high_compressed.mp4',
 '/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning/BeeBlue5_28Nov2016_Hive4_high_compressed/BeeBlue5_28Nov2016_Hive4_high_compressed.mp4',
 '/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning/BeeBlue6_28Nov2016_Hive4_high_compressed/BeeBlue6_28Nov2016_Hive4_high_compressed.mp4',
 '/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning/BeeBlue7_28Nov2016_Hive4_high_compressed/BeeBlue7_28Nov2016_Hive4_high_compressed.mp4',
 '/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning/Beeblue8_29Nov2016_Hive4_high_compress

In [115]:
# load a video
ii = 7
aa = vidPaths[ii]
cap = cv2.VideoCapture(aa)
numFr = int(round(cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT)))
print aa
print 'numFrames = ' + str(numFr)

/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning/BeeBlue_22Nov2016_Hive4_initial_compressed/BeeBlue_22Nov2016_Hive4_initial_compressed.mp4
numFrames = 50


In [116]:
cap = cv2.VideoCapture(aa)
ctr = 0

# reset frame num to zero
cap.set(cv2.cv.CV_CAP_PROP_FRAME_COUNT, 0) 

frames = []

while(ctr < numFr):  # check !
    # capture frame-by-frame
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    frames.append(gray)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    print ctr
    ctr += 1
# When everything is done release the capture
cap.release()
cv2.destroyAllWindows()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [117]:
plt.clf()
io.imshow(frames[0])

In [118]:
frames_arr = np.array(frames)

In [119]:
frames_arr.shape

(50, 1024, 1280)

In [120]:
bkg = np.median(frames_arr,axis = 0)

In [121]:
bkg.shape

(1024, 1280)

In [122]:
plt.clf()
io.imshow(bkg, cmap = "Greys")

In [123]:
# make bkg-subtracted images
bkg_sub = np.abs(frames_arr - bkg )

In [124]:
bkg_sub.shape

(50, 1024, 1280)

In [127]:
imnum = 4

In [128]:
imnum +=1

plt.clf()
io.imshow(frames[imnum])

In [129]:
# this might be the way to go

normimg = bkg_sub[imnum] / 255.0

resized_image = cv2.resize(normimg, (128, 102)) 
plt.clf()
io.imshow(resized_image)

In [130]:

plt.clf()
io.imshow(bkg_sub[imnum], cmap = "Greys")

In [131]:
# convert to BW
im_bw = (bkg_sub[imnum] > 10) * 1.0

In [132]:
im_bw.shape

(1024, 1280)

In [133]:
plt.clf()
io.imshow(im_bw)

In [134]:
# clean up image
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
erosion = cv2.erode(im_bw,kernel,iterations = 1)
plt.clf()
io.imshow(erosion)

In [135]:
dilation = cv2.dilate(erosion, kernel, iterations = 1)


plt.clf()
io.imshow(dilation)

In [136]:
# # crop image
# def submatrix(arr):
#     x, y = np.nonzero(arr)
#     # Using the smallest and largest x and y indices of nonzero elements, 
#     # we can find the desired rectangular bounds.  
#     # And don't forget to add 1 to the top bound to avoid the fencepost problem.
#     return arr[x.min():x.max()+1, y.min():y.max()+1]

# cpdImg = submatrix(dilation)



In [137]:
# reduce resolution

resized_image = cv2.resize(dilation, (128, 102)) 
plt.clf()
io.imshow(resized_image)

In [138]:
# this might be the way to go

normimg = bkg_sub[imnum] / 255.0

resized_image = cv2.resize(normimg, (128, 102)) 
plt.clf()
io.imshow(resized_image)


In [176]:
# do clustering

os.chdir("/Users/callinswitzer/Desktop/imgExamples")

smallFrames = []

for ii in range(numFr):
    normimg = bkg_sub[ii] 
    norming2 = normimg[200:, :] # remove portion with text

    resized_image = cv2.resize(norming2, (128, 82)) 
    smallFrames.append(resized_image)
    


In [178]:
plt.clf()
norming2 = normimg[200:, :]
io.imshow(norming2)
norming2.shape

(824, 1280)

In [179]:
plt.clf()
io.imshow(smallFrames[0])

In [180]:
smallFrames = np.array(smallFrames)
smallFrames.shape

(50, 82, 128)

In [181]:
# reshape array for machine learning -- each row is a photo
fr_resh = np.reshape(smallFrames, (numFr, 82*128))
fr_resh.shape


(50, 10496)

In [182]:
# knn
from sklearn.cluster import KMeans


In [248]:
X = fr_resh
X = preprocessing.scale(X) # may comment out
kmeans = KMeans(n_clusters=5, random_state=0).fit(X)
kmeans.labels_

array([1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=int32)

In [249]:
from sklearn.decomposition import PCA
from sklearn import preprocessing

In [250]:
X.std(axis=0)

np.histogram(X[1,:])

(array([   1,    7, 1069, 6414, 1328,  838,  690,  106,   26,   17]),
 array([-3.88553395, -2.79698056, -1.70842716, -0.61987377,  0.46867963,
         1.55723302,  2.64578642,  3.73433981,  4.82289321,  5.9114466 ,  7.        ]))

In [251]:
# normalize X matrix
#X = preprocessing.scale(X)

In [252]:
pca = PCA(n_components=3)
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [253]:
X2 = pca.transform(X)
y = kmeans.labels_


In [254]:
X2.shape

(50, 3)

In [262]:
plt.clf()

plt.scatter(X2[:, 1], X2[:,2], c=y, s= 100)

n = range(numFr)
for i, txt in enumerate(n):
    plt.annotate(txt, (X2[:, 1][i] + 5, X2[:,2][i] + 5))

    
plt.xlabel("PC2")
plt.ylabel("PC3")

plt.tight_layout()

plt.savefig("pca_numbered_noscale_color_manyBehaviors_pc23.png")

In [270]:
plt.clf()

plt.scatter(X2[:, 0], X2[:,1], c=y, s= 100)

n = range(numFr)
for i, txt in enumerate(n):
    plt.annotate(txt, (X2[:, 0][i] + 5, X2[:,1][i] + 5))

plt.tight_layout()

plt.savefig("pca_numbered_noscale_color_manyBehaviors.png")

In [257]:
plt.clf()
plt.scatter(X2[:, 0], X2[:,1], c=y)
plt.tight_layout()
plt.savefig("pca_small_colors_noscale.png")
plt.show()

In [273]:
plt.clf()
ii = 3
io.imshow(frames[ii])
plt.savefig(str(ii) + ".png")

In [404]:
plt.close("all")

In [287]:
resized_image = cv2.resize(normimg, (128, 102)) 
plt.clf()
io.imshow(resized_image)


In [289]:
cv2.imwrite(str(ii) + ".png", normimg)
    

True

In [239]:
from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

In [240]:
mds = MDS(n_components=2, max_iter=3000, eps=1e-9, random_state=123,
                   dissimilarity="precomputed", n_jobs=1)

In [241]:
similarities = euclidean_distances(X)

In [242]:
pos = mds.fit(similarities).embedding_

nmds = MDS(n_components=2, metric=False, max_iter=3000, eps=1e-12,
                    dissimilarity="precomputed", random_state=123, n_jobs=1,
                    n_init=1)
npos = nmds.fit_transform(similarities, init=pos)


In [243]:
# Rescale the data
pos *= np.sqrt((X ** 2).sum()) / np.sqrt((pos ** 2).sum())
npos *= np.sqrt((X ** 2).sum()) / np.sqrt((npos ** 2).sum())

# Rotate the data
clf = PCA(n_components=2)
X = clf.fit_transform(X)

pos = clf.fit_transform(pos)

npos = clf.fit_transform(npos)

fig = plt.figure(1)
ax = plt.axes([0., 0., 1., 1.])



In [247]:
plt.clf()
s = 100
plt.scatter(npos[:, 0], npos[:, 1], color='darkorange', s=s, lw=0, label='NMDS')
plt.legend(scatterpoints=1, loc='best', shadow=False)
plt.tight_layout()